<a href="https://colab.research.google.com/github/data-better/ASL/blob/master/11%EA%B0%95_date_formatter_att_keras_w_embedding_iynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import time
import datetime
from datetime import datetime

tf.enable_eager_execution() 

Using TensorFlow backend.


In [ ]:
def padding(chars, maxlen):
    if len(chars) < maxlen:
        return chars + ' ' * (maxlen - len(chars))
    else:
        return chars[:maxlen]

def gen_date():
    rnd = int(np.random.uniform(low = 1000000000, high = 2000000000))
    timestamp = datetime.fromtimestamp(rnd)
    return str(timestamp.strftime('%Y-%B-%d %a')) # '%Y-%B-%d %H:%M:%S'

def format_date(x):
    return str(datetime.strptime(x, '%Y-%B-%d %a').strftime('%m/%d/%Y, %A')).lower() #'%H%M%S:%Y%m%d'

N = 10000
N_train = int(N * .9)
N_validation = N - N_train

in_seq_len = 32
out_seq_len = 32

added = set()
questions = []
answers = []
answers_y = []

while len(questions) < N:
    if len(questions) % 1000 == 0:
        print('i = {}'.format(len(questions)))
    a = gen_date()
    if a in added:
        continue
    question = '[{}]'.format(a)
    answer = '[' + str(format_date(a)) + ']'
    answer = padding(answer, out_seq_len)
    answer_y = str(format_date(a)) + ']'
    answer_y = padding(answer_y, out_seq_len)
    
    added.add(a)
    questions.append(question)
    answers.append(answer)
    answers_y.append(answer_y)

chars = list(set(''.join(questions)))
chars.extend(['[', ']']) # Start and End of Expression
chars.extend(list(set(''.join(answers))))
chars = np.sort(list(set(chars)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

i = 0
i = 1000
i = 2000
i = 3000
i = 4000
i = 5000
i = 6000
i = 7000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 8000
i = 9000


In [ ]:
char_indices['[']

24

In [ ]:
print(questions[:10])
print(answers[:10])

['[2023-May-05 Fri]', '[2030-July-14 Sun]', '[2015-August-28 Fri]', '[2024-May-27 Mon]', '[2031-June-04 Wed]', '[2028-February-28 Mon]', '[2012-September-18 Tue]', '[2010-November-18 Thu]', '[2013-November-20 Wed]', '[2023-March-10 Fri]']
['[05/05/2023, friday]            ', '[07/14/2030, sunday]            ', '[08/28/2015, friday]            ', '[05/27/2024, monday]            ', '[06/04/2031, wednesday]         ', '[02/28/2028, monday]            ', '[09/18/2012, tuesday]           ', '[11/18/2010, thursday]          ', '[11/20/2013, wednesday]         ', '[03/10/2023, friday]            ']


In [ ]:
# Src seq
X = np.zeros((len(questions), in_seq_len), dtype=np.integer)
# Trg seq
Y = np.zeros((len(questions), out_seq_len), dtype=np.integer)
# Delayed trg seq
Z = np.zeros((len(questions), out_seq_len), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t] = char_indices[char]
    for t, char in enumerate(answers[i]):
        Y[i, t] = char_indices[char]
    for t, char in enumerate(answers_y[i]):
        Z[i, t] = char_indices[char]
    
X_train, X_validation, Y_train, Y_validation, Z_train, Z_validation = \
    train_test_split(X, Y, Z, train_size=N_train)

In [ ]:
Z_train[0]

array([ 5,  5,  3,  6,  7,  3,  6,  4,  5,  6,  1,  0, 31, 40, 34, 29, 26,
       46, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [ ]:
def gen_test(N):
    q = []
    y = []
    
    for i in range(N):
        question = gen_date()
        answer_y = format_date(question)
        q.append(question)
        y.append(answer_y)
    return(q,y)

In [ ]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [ ]:
tf.test.is_gpu_available()

True

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.CuDNNGRU(units,
                      return_sequences = True,
                      return_state = True,
                      recurrent_initializer = 'glorot_uniform'
                      )
    
  def call(self, x, hidden):
    x = self.embedding(x)
    #print('x in Encoder after embedding = {}'.format(x.shape))
    output, state = self.gru(x, initial_state = hidden)
    return output, state
  
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.enc_units))

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
    super(Decoder, self).__init__()
    self.batch_size = batch_size
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.CuDNNGRU(units,
                      return_sequences = True,
                      return_state = True,
                      recurrent_initializer = 'glorot_uniform'
                      )
    self.fc = tf.keras.layers.Dense(vocab_size)
    
    # used for attention
    self.W1 = tf.keras.layers.Dense(self.dec_units)
    self.W2 = tf.keras.layers.Dense(self.dec_units)
    self.V = tf.keras.layers.Dense(1)
    
  def call(self, x, hidden, enc_output):
    # Process single token at a time
    #enc_output: T* enc_units
    #hidden: hidden in decoder
    # Consider T as batch
    #W1: enc_units * dec_unit
    #W2: hidden * dec_unit
    #V: dec_unit * 1
    #attntion_weights: T * 1
    #enc_output: enc_units * T
    
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
    # score: batch_size * T
    attention_weights = tf.nn.softmax(score, axis = 1)
    # attention_weights: batch_size * T
    
    context_vector = attention_weights * enc_output
    context_vector = tf.reduce_sum(context_vector, axis = 1)
    #context_vector: batch_size * hidden_size
    x = self.embedding(x)
    # x: batch_size * 1 * embedding_dim
    #print('context vector size = {}'.format(context_vector.shape))
    #print('x vector size = {}'.format(x.shape))
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis = -1)
    # x: batch_size * 1 * (hidden_size + embedding_dim)
    
    output, state = self.gru(x)
    #output: batch_size * 1* hidden_size
    output = tf.reshape(output, (-1, output.shape[2]))
    #output: batch_size * hidden_size
    x = self.fc(output)
    #output: batch_size * vocab_size
    
    return x, state, attention_weights
  
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.dec_units))

In [ ]:
def evaluate(sentence, encoder, decoder, char_indices, indices_char, max_len_inp, max_len_targ, hidden_units):
  attention_plot = np.zeros((max_len_targ, max_len_inp))
  inp = np.zeros((1, max_len_inp), dtype=np.integer)
  for t, char in enumerate(sentence):
    inp[0, t] = char_indices[char]
  inp = tf.convert_to_tensor(inp)
  
  result = ''
  hidden = [tf.zeros((1, hidden_units))]
  enc_out, enc_hidden = encoder(inp, hidden)
  dec_hidden = enc_hidden
  dec_inp = tf.expand_dims([char_indices['[']], 0)
  
  for t in range(max_len_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_inp, dec_hidden, enc_out)
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()
    
    predicted_id = tf.argmax(predictions[0]).numpy()
    
    result += indices_char[predicted_id]
    if indices_char[predicted_id] == ']':
      return result, sentence, attention_plot
    
    dec_inp = tf.expand_dims([predicted_id], 0)
    
  return result, sentence, attention_plot

In [ ]:
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize = (10, 10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap = 'viridis')
  fontdict = {'fontsize': 14}
  
  ax.set_xticklabels([''] + sentence, fontdict = fontdict, rotation = 90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict = fontdict)
  plt.show()

In [ ]:
def evaluate_some(n, encoder, decoder, char_indices, indices_char, max_len_inp, max_len_targ, hidden_units):
  q, y = gen_test(n)
  for i in range(n):
    p, _, _ = evaluate(q[i], encoder, decoder, char_indices, indices_char, max_len_inp, max_len_targ, hidden_units)
    p = p.strip()
    iscorr = 1 if p == y[i] else 0
    if iscorr == 1:
      print(colors.ok + '☑' + colors.close, end=' ')
    else:
      print(colors.fail + '☒' + colors.close, end=' ')
    print("{} = {}({})".format(q[i], p, y[i]))

In [ ]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 32
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(chars)
vocab_tar_size = len(chars)

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = real, logits = pred) * mask
  return tf.reduce_mean(loss_)

In [ ]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer = optimizer,
                                 encoder = encoder,
                                 decoder = decoder)

In [ ]:
import time

EPOCHS = 200

for epoch in range(EPOCHS):
  start = time.time()
  hidden = encoder.initialize_hidden_state()
  total_loss = 0
  
  for batch in range(N_BATCH):
    loss = 0
    idx = np.arange(batch * BATCH_SIZE, (batch + 1) * BATCH_SIZE)
    inp = X_train[idx, :]
    targ = Y_train[idx, :]
    #print('inp shape = {}'.format(inp.shape))
    with tf.GradientTape() as tape:
      enc_output, enc_hidden = encoder(inp, hidden)
      dec_hidden = enc_hidden
      dec_input = tf.expand_dims([char_indices['[']] * BATCH_SIZE, 1)
      
      # Teacher forcing
      
      for t in range(1, targ.shape[1]):
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
        loss += loss_function(targ[:, t], predictions)
        dec_input = tf.expand_dims(targ[:, t], 1)
      
    batch_loss = (loss / int(targ.shape[1]))
    total_loss += batch_loss
      
    variables = encoder.variables + decoder.variables
    gradients = tape.gradient(loss, variables)
      
    optimizer.apply_gradients(zip(gradients, variables))
      
    #if batch % 10 == 0:
    #  print('Epoch {} Batch {} Loss: {:.6f}'.format(epoch +1, 
    #                                            batch, 
    #                                            batch_loss.numpy()))
        
  if (epoch + 1) % 10 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    evaluate_some(10, encoder, decoder, char_indices, indices_char, 32, 32, units)
        
  print('Epoch {} Loss {:.6f}, Elapsed_time ={:.2f} sec'.format(epoch + 1, 
                                                                total_loss / N_BATCH, 
                                                                time.time() - start))

Epoch 1 Loss 0.610379, Elapsed_time =217.43 sec
Epoch 2 Loss 0.204183, Elapsed_time =218.60 sec
Epoch 3 Loss 0.007149, Elapsed_time =218.91 sec
Epoch 4 Loss 0.000253, Elapsed_time =218.39 sec
Epoch 5 Loss 0.000108, Elapsed_time =219.30 sec
Epoch 6 Loss 0.018136, Elapsed_time =219.00 sec
Epoch 7 Loss 0.000247, Elapsed_time =218.32 sec
Epoch 8 Loss 0.000083, Elapsed_time =218.11 sec
Epoch 9 Loss 0.000048, Elapsed_time =217.30 sec
☒ 2027-May-26 Wed = 05/26/2027,dnednesday](05/26/2027, wednesday)
☒ 2008-March-28 Fri = 03/28/2008,iday](03/28/2008, friday)
☒ 2019-July-11 Thu = 07/11/2019,](07/11/2019, thursday)
☒ 2020-December-02 Wed = 12/02/2020,](12/02/2020, wednesday)
☒ 2032-March-28 Sun = 03/28/2032,turday](03/28/2032, sunday)
☒ 2016-January-08 Fri = 01/08/2016,iday](01/08/2016, friday)
☒ 2012-April-22 Sun = 04/22/2012,](04/22/2012, sunday)
☒ 2008-February-23 Sat = 02/23/2008,](02/23/2008, saturday)
☒ 2012-July-05 Thu = 07/05/2012,](07/05/2012, thursday)
☒ 2012-June-26 Tue = 06/26/2012,]

In [ ]:
q, y = gen_test(1)
result, sentence, _ = evaluate(q[0], encoder, decoder, char_indices, indices_char,32, 32)
print('pred = {}, inp = {}'.format(result, sentence))

NameError: ignored

In [ ]:
evaluate_some(3, encoder, decoder, char_indices, indices_char,32, 32)